In [2]:
#import Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
#get DataSet
fruits = pd.read_csv("fruit_data_with_colours.csv")

In [4]:
fruits.head(5)

,fruit_label,fruit_name,fruit_subtype,mass,width,height,color_score
0,1,apple,granny_smith,192,8.4,7.3,0.55
1,1,apple,granny_smith,180,8.0,6.8,0.59
2,1,apple,granny_smith,176,7.4,7.2,0.60
3,2,mandarin,mandarin,86,6.2,4.7,0.80
4,2,mandarin,mandarin,84,6.0,4.6,0.79


In [5]:
print(min(fruits['width']))
Fruit_name =  dict(zip(fruits.fruit_label.unique(),fruits.fruit_name.unique() ))

5.8


In [6]:
print(type(Fruit_name))
print(Fruit_name)

<class 'dict'>
{1: 'apple', 2: 'mandarin', 3: 'orange', 4: 'lemon'}


In [7]:
#get independent_variable (Features)
x = fruits[['mass','width','height']]

In [8]:
x.head(5)

,mass,width,height
0,192,8.4,7.3
1,180,8.0,6.8
2,176,7.4,7.2
3,86,6.2,4.7
4,84,6.0,4.6


In [9]:
#get dependent_variable (Response)
y = fruits['fruit_label']
y.head()

0    1
1    1
2    1
3    2
4    2
Name: fruit_label, dtype: int64

In [10]:
#Split data 
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42)

In [11]:
#Classifier using KNN
from sklearn.neighbors import KNeighborsClassifier

In [12]:
score = {}
for k in range(1,16,2):
    knn = KNeighborsClassifier(n_neighbors=k)
    #train_data
    knn.fit(x_train,y_train)
    #test_data
    score[k]=knn.score(x_test,y_test)
max_score = max(score,key=score.get)

In [13]:
score

{1: 0.5333333333333333,
 3: 0.6,
 5: 0.5333333333333333,
 7: 0.6,
 9: 0.5333333333333333,
 11: 0.4666666666666667,
 13: 0.4666666666666667,
 15: 0.4}

In [14]:
max_score

3

In [15]:
#Get predicted response 
y_predict = knn.predict(x_test)

In [16]:
y_predict

array([1, 4, 3, 1, 1, 4, 3, 1, 1, 3, 4, 4, 1, 1, 1], dtype=int64)

In [17]:
sample = knn.predict([[120,5.3,8.5]])

In [18]:
sample

array([4], dtype=int64)

In [19]:
sample_name_predicted = Fruit_name[sample[0]]
print(sample_name_predicted)

lemon
